In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 15.9MB/s]


In [3]:
city = 'Spokane'

# Create empty list to store scraped data
datatoget = []

for i in range(1,9):

    # Set url
    url = f'https://spokane.craigslist.org/search/spokane-wa/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))


    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n================')
    print(f'This is page {i}')
    print('================')

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):

        # Scrape dates
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan

        # Scrape titles
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']

        try:
        # Scrape prices
            prices = link.find_all('span', class_='priceinfo')[0].text
        except:
            prices = np.nan

        # Scrape neighborhoods
        try:
            neighborhoods = link.find('div', class_='supertitle').text

        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)

        # Ensures that the server does not get overloaded with requests
        sleep(randint(3,5))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')


        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan

        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan

        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan

        try:
            amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
            # List comprehension used to make sure no empty strings get added to the list
            all_amenities = [am for am in amenities if am != '']
        except:
            amenities = np.nan

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This is page 1

Listing Number: 1
Square Footage: 1077
Price: $1,529
Title: Check Out The Fastest Growing Development In Liberty Lake!
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Liberty Lake
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/liberty-lake-check-out-the-fastest/7579535768.html
Amenities: ['air conditioning', 'application fee details: app $60, admin $200, security deposit $300', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'detached garage', 'rent period: monthly']

Listing Number: 2
Square Footage: 817
Price: $1,402
Title: Rare Opening: Ground Floor With Attached Garage!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: not available
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/liberty-lake-rare-opening-ground-floor/7579534938.html
Amenities: ['air conditioning', 'application fee details: app $60, admin $200, security deposit $300', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartmen


Listing Number: 18
Square Footage: 917
Price: $1,565
Title: THE PERFECT ONE BEDROOM! AMAZING DEAL!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Spokane Valley
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-the-perfect-one-bedroom-amazing/7579523660.html
Amenities: ['air conditioning', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 19
Square Footage: 719
Price: $1,276
Title: 🌌Sparkling pool, Fitness center, Resident parking🌌
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 2808 South Assembly Road, Spokane, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-sparkling-pool-fitness-center/7576223417.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport', 'rent period: monthly']

Listing Number: 20
Square Footage: 719
Price: $1,276
Title: 🙊🐵 TAKE YOUR PICK OF THESE BEAUTIFUL 1BED 1BATH HOMES🐵🙊
Date: 1/17
Number of


Listing Number: 36
Square Footage: 719
Price: $1,276
Title: 🧏‍♀️🛀Quite Community, Private Balcony, Move In Specials 🛀🧏‍
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 2808 South Assembly Road, Spokane, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-quite-community-private-balcony/7579495254.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport', 'rent period: monthly']

Listing Number: 37
Square Footage: 600
Price: $1,150
Title: Clubhouse, Fitness Center, Patio/Balcony
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: 16102 East Broadway Avenue, Spokane Valley, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/veradale-clubhouse-fitness-center-patio/7579490967.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 38
Square Fo


Listing Number: 54
Square Footage: 1140
Price: $1,566
Title: 2️⃣❎2️⃣ LAST GROUND FLOOR AVAILABLE!!! 2️⃣❎2️⃣
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: 7024 North Colton Street, Spokane, WA
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/spokane-22-last-ground-floor-available/7578712826.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 55
Square Footage: 795
Price: $1,100
Title: Nice 2 Bedroom
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Spokane Valley
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-nice-bedroom/7577612208.html
Amenities: ['application fee details: $45 credit, criminal,employment and landlord check', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 56
Square F


Listing Number: 72
Square Footage: 1140
Price: $1,566
Title: 🌴🥥 WE’RE COCONUTS ABOUT THIS APARTMENT!!! 🥥🌴
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: not available
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/spokane-were-coconuts-about-this/7577138090.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 73
Square Footage: 755
Price: $1,400
Title: *See Special* 1 Bed 1 Bath Home AVAILABLE NOW at Beau West Apartments!
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: 7410 South Hayford Road, Spokane, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-see-special-bed-bath-home/7577791447.html
Amenities: ['EV charging', 'air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 

...


Listing Number: 34
Square Footage: 1700
Price: $2,200
Title: Buy or Rent-to-Own 2607 W Dean Ave
Date: 1/3
Number of Bedrooms: 4br
Neighborhood: Spokane
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-buy-or-rent-to-own-2607-dean-ave/7574460081.html
Amenities: ['house', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 35
Square Footage: 1268
Price: $2,500
Title: Buy or Rent to Own 1908 E Cataldo Ave
Date: 1/3
Number of Bedrooms: 3br
Neighborhood: Spokane
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-buy-or-rent-to-own-1908-cataldo/7574457707.html
Amenities: ['house', 'w/d hookups', 'detached garage', 'rent period: monthly']

Listing Number: 36
Square Footage: 700
Price: $1,000
Title: 1 bedroom bottom level move in ready
Date: 1/3
Number of Bedrooms: 1br
Neighborhood: Medical Lake
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/fairchild-air-force-base-bedroom-bottom/7574472503.html
Amenitie


Listing Number: 54
Square Footage: 1700
Price: $1,450
Title: NEW PRICE - Spacious 2+ bedroom in Historic Home
Date: 1/2
Number of Bedrooms: 2br
Neighborhood: Browne’s Addition
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-new-price-spacious-bedroom-in/7574316837.html
Amenities: ['application fee details: $40 background check fee', 'apartment', 'laundry on site', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 55
Square Footage: 719
Price: $1,276
Title: ⚖Friendly on-site management, Non smoking ⚖
Date: 1/2
Number of Bedrooms: 1br
Neighborhood: 2808 South Assembly Road, Spokane, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-friendly-on-site-management-non/7571693889.html
Amenities: ['apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 56
Square Footage: 1190
Price: $1,862
Title: 🥂🕊🥂 Cheers To A New Home For The New Year 🥂🕊🥂
Date: 1/2
Number of Bedrooms: 3br
Neighborhood: 2808 S


Listing Number: 73
Square Footage: 1250
Price: $1,677
Title: 👖👕👗 HUGE WALK-IN CLOSETS!!! 👗👕👖
Date: 1/1
Number of Bedrooms: 3br
Neighborhood: 7024 North Colton Street, Spokane, WA
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/spokane-huge-walk-in-closets/7571512337.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 74
Square Footage: 875
Price: $1,468
Title: 🐱‍👓🐱‍👤 Pet Friendly 2 Bed 1 Bath 🐱‍👤🐱‍👓
Date: 1/1
Number of Bedrooms: 2br
Neighborhood: 2808 South Assembly Road, Spokane, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-pet-friendly-bed-bath/7571526897.html
Amenities: ['apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 75
Square Footage: 1250
Price: $1,677
Title: 🌬️☁️☀️ VAULTED CEILINGS!!! ☀️☁️🌬️
Date: 1/1
Number of Bedrooms: 3br
Neighborhood: 7024 Nor


Listing Number: 92
Square Footage: 875
Price: $1,468
Title: 🧺1️⃣👀 In Unit Washer/ Dryer, Pet Friendly 2 Bed 1 Bath 👀1️⃣🧺
Date: 12/31
Number of Bedrooms: 2br
Neighborhood: 2808 South Assembly Road, Spokane, WA
Number of Bathrooms: 1
Link: https://spokane.craigslist.org/apa/d/spokane-in-unit-washer-dryer-pet/7571241638.html
Amenities: ['apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 93
Square Footage: not available
Price: $1,473
Title: Rare 2 Bedroom- ONE MONTH FREE!!!
Date: 12/31
Number of Bedrooms: 2br
Neighborhood: not available
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/spokane-rare-bedroom-one-month-free/7573700534.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 94
Square Footage: not available
Price: $1,256
Title: Rare 1 Bedroom!
Date: 12/31
Number of Bedrooms: 1br
Neighborhood: not availa


Listing Number: 111
Square Footage: 1140
Price: $1,566
Title: 🌠 YOUR WISHES WILL COME TRUE HERE!!! 🌠
Date: 12/31
Number of Bedrooms: 2br
Neighborhood: not available
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/spokane-your-wishes-will-come-true-here/7571158567.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 112
Square Footage: 1025
Price: $1,466
Title: 👌🏽🙆🏽‍♀️ PICK ME!!! 🙆🏽‍♂️👍🏽
Date: 12/31
Number of Bedrooms: 2br
Neighborhood: not available
Number of Bathrooms: 2
Link: https://spokane.craigslist.org/apa/d/spokane-pick-me/7571158292.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 113
Square Footage: 1140
Price: $1,566
Title: 🏘️🌌 SPACIOUS FLOORPLANS!!! 🌌🏘️


In [ ]:
x()

In [5]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/17,Check Out The Fastest Growing Development In L...,https://spokane.craigslist.org/apa/d/liberty-l...,"$1,529",2br,1077,Liberty Lake,2,"[air conditioning, application fee details: ap..."
1,1/17,Rare Opening: Ground Floor With Attached Garage!,https://spokane.craigslist.org/apa/d/liberty-l...,"$1,402",1br,817,NaN,1,"[air conditioning, application fee details: ap..."
2,1/17,AMAZING PET FRIENDLY ONE BEDROOM!,https://spokane.craigslist.org/apa/d/spokane-a...,"$1,515",1br,922,Spokane Valley,1,"[air conditioning, cats are OK - purrr, dogs a..."
3,1/17,One Month FREE! Call Today To Learn More & Sch...,https://spokane.craigslist.org/apa/d/liberty-l...,"$1,530",2br,1025,Liberty Lake,2,"[air conditioning, application fee details: $6..."
4,1/17,"Covered Parking, Carport, Cable Ready",https://spokane.craigslist.org/apa/d/veradale-...,"$1,255",1br,727,"16102 East Broadway Avenue, Spokane Valley, WA",1,"[air conditioning, cats are OK - purrr, dogs a..."
...,...,...,...,...,...,...,...,...,...
955,12/30,Sit Back. Relax. Your New View Awaits.,https://spokane.craigslist.org/apa/d/spokane-s...,"$1,473",2br,NaN,NaN,2,"[air conditioning, cats are OK - purrr, dogs a..."
956,12/30,Welcome Home!,https://spokane.craigslist.org/apa/d/spokane-w...,"$1,256",1br,NaN,NaN,1,"[air conditioning, cats are OK - purrr, dogs a..."
957,12/30,Studio of Your Dreams!,https://spokane.craigslist.org/apa/d/spokane-s...,"$1,255",0br,NaN,NaN,1,"[air conditioning, cats are OK - purrr, dogs a..."
958,12/30,Furnished Studio Apartment,https://spokane.craigslist.org/apa/d/greenacre...,"$1,100",0br,NaN,Spokane Valley,1,"[air conditioning, cats are OK - purrr, dogs a..."


In [24]:
# Exporting data to csv
df.to_csv(f"{city.replace(' ', '_')}_data_raw.csv", index=False)